In [19]:
import json

with open("yelp1.json", 'r') as data_file:
    data = json.load(data_file)

new_file = []
for cuisine, rest_info in data.items():
    tmp = rest_info
    for rest in tmp:
        rest["cuisine"] = cuisine
    new_file.append(tmp)
    
with open("data.json", 'w') as data_file:
    data = json.dump(new_file, data_file)

In [31]:
with open("data.json", "r") as data_file:
    data = json.load(data_file)

new_file = []
for element in data:
    for rest in element:
        new_file.append(rest)

with open("yelp1.json", "w") as data_file:
    json.dump(new_file, data_file)

In [46]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal
import datetime

dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

### create table ###

table = dynamodb.create_table(
    TableName='yelp-restaurants',

    KeySchema = [
        {
            'AttributeName': 'cuisine',
            'KeyType': 'HASH' # The values of two partition keys could be the same
        },
        {
            'AttributeName': 'id',
            'KeyType': 'RANGE'
        }
    ],

    AttributeDefinitions=[
        {
            'AttributeName': 'cuisine',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'id',
            'AttributeType': 'S'
        }
    ],

    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)

print("Table status:", table.table_status)


Table status: CREATING


In [50]:
### import JSON to items ###
from datetime import *
table = dynamodb.Table('yelp-restaurants')

with open("yelp1.json", 'r') as json_file:
    data = json.load(json_file, parse_float = decimal.Decimal)
    for restaurant in data:
        cuisine = restaurant["cuisine"]
        ID = restaurant["id"]
        name = restaurant["name"]
#         price = restaurant["price"]
        coord = "coordinates are not available" if "coordinates" not in restaurant else restaurant["coordinates"]
        location = "location is not available" if "location" not in restaurant else restaurant["location"] 
        cate = "categories are not available" if "categories" not in restaurant else restaurant["categories"]
        time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        table.put_item(
            Item = {
                'cuisine': cuisine,
                'id': ID,
                'name': name,
                'coordinates': coord,
                'location': location,
                'categories': cate,
                'insertedAtTimestamp': time
            }       
        )

In [26]:
import json

with open("yelp1.json", "r") as data_file:
    data = json.load(data_file)


i = 1
new_file = []
for element in data:
    tmp = element
    if "price" in tmp:
        tmp.pop("price")
    if "coordinates" in tmp:
        tmp.pop("coordinates")
    if "location" in tmp:
        tmp.pop("location")
    if "categories" in tmp:
        tmp.pop("categories")
    dic_inst = {"index": {"_index": "restaurants", "_type": "Restaurant", "_id": str(i)}}
    new_file.append(dic_inst)
    new_file.append(tmp)
    i += 1

with open("yelp_es.json", "w") as data_file:
    json.dump(new_file, data_file)
